In [18]:
import gzip
import numpy as np
from requests import get
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard
from time import time
from keras.models import Sequential

In [19]:
def read_MNIST(images_path: str, labels_path: str):
    with gzip.open(labels_path, 'rb') as lf:
        labels = np.frombuffer(lf.read(), dtype=np.uint8, offset=8)
    with gzip.open(images_path,'rb') as iF:
        length = len(labels)
        features = np.frombuffer(iF.read(), dtype=np.uint8, offset=16) \
                        .reshape(length, 784) \
                        .reshape(length, 28, 28, 1)
        
    return features, labels

In [20]:
train = {}
test = {}

train['features'], train['labels'] = read_MNIST('train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz')
test['features'], test['labels'] = read_MNIST('t10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz')

In [21]:
print('Training Images:', train['features'].shape[0])
print('Test Images:', test['features'].shape[0])

Training Images: 60000
Test Images: 10000


In [22]:
model = keras.Sequential()
model.add(layers.Conv2D(filters = 6, kernel_size = (3,3), activation = 'relu', input_shape = (32, 32, 1)))
model.add(layers.AveragePooling2D())
model.add(layers.Conv2D(filters = 16, kernel_size = (3, 3), activation = "relu"))
model.add(layers.AveragePooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(units=120, activation='relu'))
model.add(layers.Dense(units=84, activation='relu'))
model.add(layers.Dense(units=10, activation = 'softmax'))

In [23]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer = 'adam', metrics = ['accuracy'])

In [24]:
train['features']      = np.pad(train['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
test['features']       = np.pad(test['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_train, y_train = train['features'], to_categorical(train['labels'])
x_test, y_test = test['features'], to_categorical(test['labels'])
x_train[0].shape

(32, 32, 1)

In [25]:
EPOCHS = 10 
batch_size = 128 

In [26]:
train_generator = ImageDataGenerator().flow(x_train, y_train, batch_size=batch_size)

In [27]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=EPOCHS, validation_split=0.1)

Epoch 1/10
422/422 [==============================] - 6s 14ms/step - loss: 0.9144 - accuracy: 0.8827 - val_loss: 0.1297 - val_accuracy: 0.9610
Epoch 2/10
422/422 [==============================] - 6s 15ms/step - loss: 0.1153 - accuracy: 0.9660 - val_loss: 0.0751 - val_accuracy: 0.9800
Epoch 3/10
422/422 [==============================] - 7s 16ms/step - loss: 0.0746 - accuracy: 0.9772 - val_loss: 0.0675 - val_accuracy: 0.9813
Epoch 4/10
422/422 [==============================] - 7s 16ms/step - loss: 0.0574 - accuracy: 0.9822 - val_loss: 0.0649 - val_accuracy: 0.9837
Epoch 5/10
422/422 [==============================] - 7s 17ms/step - loss: 0.0460 - accuracy: 0.9856 - val_loss: 0.0677 - val_accuracy: 0.9815
Epoch 6/10
422/422 [==============================] - 7s 17ms/step - loss: 0.0375 - accuracy: 0.9882 - val_loss: 0.0497 - val_accuracy: 0.9863
Epoch 7/10
422/422 [==============================] - 7s 18ms/step - loss: 0.0333 - accuracy: 0.9891 - val_loss: 0.0649 - val_accuracy: 0.9848

In [28]:
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 1s 2ms/step - loss: 0.0425 - accuracy: 0.9865
Test loss: 0.0424901507794857
Test accuracy: 0.9865000247955322
